In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer # model to convert text into code, "sentence classification"
import numpy as np # store converted text into numbers
import pandas as pd # dataset

In [ ]:
df = pd.read_csv('./falcon_results (2).csv')

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


## new dataframe to store our data
answer_embeddings = df['answer'].apply(lambda x: model.encode(x, convert_to_tensor=True))
response_embeddings = df['model_response'].apply(lambda x: model.encode(x, convert_to_tensor=True))

embedding_df = pd.DataFrame({
    'answer_embedding': answer_embeddings,
    'model_response_embedding': response_embeddings
})

In [ ]:
embedding_df

In [ ]:
## COSINE SIMMULARITY

import torch # pytorch

answers = torch.stack(embedding_df['answer_embedding'].tolist())          # 768
responses = torch.stack(embedding_df['model_response_embedding'].tolist()) # 768

cos_sim = torch.nn.functional.cosine_similarity(answers, responses, dim=1)

embedding_df['cosine_similarity'] = cos_sim.tolist()

print("cosine similarity:", cos_sim.mean().item())

In [ ]:
## BERTScore
!pip install bert_score
from bert_score import score

P, R, F1 = score(df['answer'].tolist(), df['model_response'].tolist(), lang='en')
print(f"BERTScore F1: {F1.mean().item():.2f}")